In [27]:
import csv
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import gaussian_kde

In [28]:
#prefix 
prefix='NGC6946'

# first magnitude
mag1 = "J"
# second magnitude
mag2 = "K"
# number of mag1 images:
mag1number = 2
# number of mag2 images:
mag2number = 6

# is this ACS?
isAcs = False

#real photometry first
real_file = "%s.phot" % prefix
real_df = pd.read_csv(real_file, delim_whitespace=True, header=None) #create data frame 
real_df = real_df.iloc[:,:37] #select till relevant end of data frame (quality flag of I)

#name columns
columns=['extension','chip','x','y','chi','snr','sharp','round','maj_ax','crowd','type',
           'cts_mag1','sky_mag1','nrm_ct_rt_mag1','nrm_ct_rt_err_mag1','inst_vega_mag1','mag_mag1','mag1_err','chi_mag1','snr_mag1',
           'sharp_mag1','round_mag1','crowd_mag1','flag_mag1','ctsmag2','skymag2','nrm_ct_rt_mag2','nrm_ct_rt_err_mag2', 
           'inst_vega_mag2','mag_mag2','mag2_err','chi_mag2','snr_mag2','shar_pmag2','round_mag2','crowd_mag2','flag_mag2']

In [29]:
#assign columns and preview
real_df.columns=columns

#fix y column if not on first chip 
if (isAcs):
        real_df['y'] = real_df.apply(lambda x: x.y+2000 if x.extension>1 else x.y, axis=1)

In [30]:
#establish cuts from below line
#$5<=2.5 && $7*$7<=0.09 && $11<=2 && $20>=5 && $24==0 && $33>=5 && $37==0'' > ${TARG}.phot2")

real_cut = real_df[(real_df['chi'] < 2.5) & (real_df['sharp']*real_df['sharp'] <= 0.09) & (real_df['type'] <= 2) 
        & (real_df['snr_mag1'] >= 5) & (real_df['flag_mag1'] == 0) & (real_df['snr_mag2'] >= 5) 
        & (real_df['flag_mag2'] == 0)]



In [31]:
real_cut[real_cut.columns[1:11]][:5]

,chip,x,y,chi,snr,sharp,round,maj_ax,crowd,type
5,1,140.87,789.13,1.44,11112.4,0.003,0.025,100,0.001,1
6,1,415.23,377.82,1.05,10735.4,0.003,0.009,155,0.000,1
7,1,850.94,675.01,1.09,10303.3,-0.001,0.022,170,0.002,1
9,1,803.85,631.93,1.07,7012.7,0.012,0.010,25,0.000,1
11,1,124.91,687.23,1.43,5852.1,0.008,0.027,90,0.001,1


In [32]:
real_cut[real_cut.columns[11:24]][:5]

,cts_mag1,sky_mag1,nrm_ct_rt_mag1,nrm_ct_rt_err_mag1,inst_vega_mag1,mag_mag1,mag1_err,chi_mag1,snr_mag1,sharp_mag1,round_mag1,crowd_mag1,flag_mag1
5,74900000.0,8535.76,1.870000e-06,2.390000e-10,14.322,99.999,0.0,1.72,7805.7,0.021,0.060,0.000,0
6,69000000.0,8663.01,1.720000e-06,2.280000e-10,14.412,99.999,0.0,1.42,7526.3,0.001,0.019,0.000,0
7,62200000.0,20005.44,1.550000e-06,2.170000e-10,14.525,99.999,0.0,1.47,7146.3,0.019,0.048,0.001,0
9,26900000.0,11245.76,6.710000e-07,1.430000e-10,15.433,99.999,0.0,1.41,4694.2,-0.001,0.032,0.000,0
11,21100000.0,5727.11,5.250000e-07,1.260000e-10,15.700,99.999,0.0,1.55,4149.8,-0.001,0.046,0.000,0


In [33]:
real_cut[real_cut.columns[24:37]][:5]

,ctsmag2,skymag2,nrm_ct_rt_mag2,nrm_ct_rt_err_mag2,inst_vega_mag2,mag_mag2,mag2_err,chi_mag2,snr_mag2,shar_pmag2,round_mag2,crowd_mag2,flag_mag2
5,82000000.0,13511.32,2.720000e-06,3.440000e-10,13.912,99.999,0.0,1.33,7908.8,-0.004,0.012,0.001,0
6,76600000.0,12536.77,2.540000e-06,3.320000e-10,13.986,99.999,0.0,0.90,7654.8,0.004,0.005,0.000,0
7,72200000.0,30190.16,2.400000e-06,3.230000e-10,14.049,99.999,0.0,0.94,7418.9,-0.009,0.012,0.003,0
9,35800000.0,16070.76,1.190000e-06,2.290000e-10,14.811,99.999,0.0,0.93,5209.2,0.017,0.003,0.000,0
11,22500000.0,8507.76,7.480000e-07,1.810000e-10,15.315,99.999,0.0,1.39,4126.1,0.012,0.021,0.001,0


In [34]:
#print out real_cut to space separated file
real_cut_file = real_cut.to_csv('%s.phot2' % (prefix), sep=' ', header = True, index = False)

In [35]:
#fake photometry next
fake_file='%s.phot.fake' % prefix #select file 

In [44]:
fake_df = pd.read_csv(fake_file, delim_whitespace=True, header=None) #create data frame 

## calculate number of columns
#numberOfColumns = len(fake_df.columns)
#numberOfExtraColumns = numberOfColumns - 71
#numberOfImagesPerFilter = int(1+(numberOfExtraColumns/30))
#numberOfRelevantColumns = (numberOfImagesPerFilter-1)*4

#fake_df = fake_df.iloc[:,:(45+(numberOfRelevantColumns))] #select till relevant end of data frame (quality flag of I)

In [49]:
base_columns=['extensionIn','chipIn','xIn','yIn']
#'counts_V1','mag_V1','counts_I1', 'mag_I1','extensionOut','chipOut','xOut','yOut',
#           'chi','snr','sharp','round','maj_ax','crowd','type','cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V',
#           'inst_vega_V','mag_V','Verr','chi_V','snr_V','sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I',
#           'nrm_ct_rt_I','nrm_ct_rt_err_I', 'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I',
#           'crowd_I','flag_I'] #base columns

In [58]:
#loop to find correct number of additional photometry columns, and insert them
base_columns=['extensionIn','chipIn','xIn','yIn']
# add input mag1 columns

for counter in range(0,mag1number):
    base_columns = base_columns + [('counts_mag1_%d' % counter),('mag_mag1_%d' % counter)]
    
# add input mag1 columns

for counter in range(0,mag2number):
    base_columns = base_columns + [('counts_mag2_%d' % counter),('mag_mag2_%d' % counter)]
    
# add new set of input geometry data

base_columns = base_columns + ['extensionOut','chipOut','xOut','yOut']
base_columns = base_columns + ['chimagOut','snrmagOut','sharpmagOut','roundmagOut','maj_axmagOut','crowdmagOut','typemagOut']
base_columns = base_columns + ['countsmag1Out','skymag1Out','nrmctmag1Out','nrmcterrmag1Out','instvegamagmag1Out','magmag1Out','mag1errOut','snrmag1Out','sharpmag1Out','roundmag1Out','maj_axmag1Out','crowdmag1Out','typemag1Out']
base_columns = base_columns + ['countsmag2Out','skymag2Out','nrmctmag2Out','nrmcterrmag2Out','instvegamagmag2Out','magmag2Out','mag2errOut','snrmag2Out','sharpmag2Out','roundmag2Out','maj_axmag2Out','crowdmag2Out','typemag2Out']

# add output mag1 columns

for counter in range(0,mag1number):
    base_columns = base_columns + [('cts_mag1_%d' % counter)]
    base_columns = base_columns + [('sky_mag1_%d' % counter)]
    base_columns = base_columns + [('nrm_ct_rt_mag1_%d' % counter)]
    base_columns = base_columns + [('nrm_ct_rt_err_mag1_%d' % counter)]
    base_columns = base_columns + [('inst_vega_mag1_%d' % counter)]
    base_columns = base_columns + [('mag_mag1_%d' % counter)]
    base_columns = base_columns + [('mag1_err_%d' % counter)]
    base_columns = base_columns + [('chi_mag1_%d' % counter)]
    base_columns = base_columns + [('snr_mag1_%d' % counter)]
    base_columns = base_columns + [('sharp_mag1_%d' % counter)]
    base_columns = base_columns + [('round_mag1_%d' % counter)]
    base_columns = base_columns + [('crowd_mag1_%d' % counter)]
    base_columns = base_columns + [('flag_mag1_%d' % counter)]

for counter in range(0,mag2number):
    base_columns = base_columns + [('cts_mag2%d' % counter)]
    base_columns = base_columns + [('sky_mag2_%d' % counter)]
    base_columns = base_columns + [('nrm_ct_rt_mag1_%d' % counter)]
    base_columns = base_columns + [('nrm_ct_rt_err_mag1_%d' % counter)]
    base_columns = base_columns + [('inst_vega_mag1_%d' % counter)]
    base_columns = base_columns + [('mag_mag1_%d' % counter)]
    base_columns = base_columns + [('mag1_err_%d' % counter)]
    base_columns = base_columns + [('chi_mag1_%d' % counter)]
    base_columns = base_columns + [('snr_mag1_%d' % counter)]
    base_columns = base_columns + [('sharp_mag1_%d' % counter)]
    base_columns = base_columns + [('round_mag1_%d' % counter)]
    base_columns = base_columns + [('crowd_mag1_%d' % counter)]
    base_columns = base_columns + [('flag_mag1_%d' % counter)]

    
print(base_columns)
    
fake_df.columns=base_columns 


['extensionIn', 'chipIn', 'xIn', 'yIn', 'counts_mag1_0', 'mag_mag1_0', 'counts_mag1_1', 'mag_mag1_1', 'counts_mag2_0', 'mag_mag2_0', 'counts_mag2_1', 'mag_mag2_1', 'counts_mag2_2', 'mag_mag2_2', 'counts_mag2_3', 'mag_mag2_3', 'counts_mag2_4', 'mag_mag2_4', 'counts_mag2_5', 'mag_mag2_5', 'extensionOut', 'chipOut', 'xOut', 'yOut', 'chimagOut', 'snrmagOut', 'sharpmagOut', 'roundmagOut', 'maj_axmagOut', 'crowdmagOut', 'typemagOut', 'countsmag1Out', 'snrmag1Out', 'sharpmag1Out', 'roundmag1Out', 'maj_axmag1Out', 'crowdmag1Out', 'typemag1Out', 'countsmag2Out', 'snrmag2Out', 'sharpmag2Out', 'roundmag2Out', 'maj_axmag2Out', 'crowdmag2Out', 'typemag2Out', 'cts_mag2_0', 'sky_mag2_0', 'nrm_ct_rt_mag2_0', 'nrm_ct_rt_err_mag2_0', 'inst_vega_mag2_0', 'mag_mag2_0', 'mag2_err_0', 'chi_mag2_0', 'snr_mag2_0', 'sharp_mag2_0', 'round_mag2_0', 'crowd_mag2_0', 'flag_mag2_0', 'cts_mag2_1', 'sky_mag2_1', 'nrm_ct_rt_mag2_1', 'nrm_ct_rt_err_mag2_1', 'inst_vega_mag2_1', 'mag_mag2_1', 'mag2_err_1', 'chi_mag2_1', '

ValueError: Length mismatch: Expected axis has 161 elements, new values have 201 elements

In [10]:
# correct Y position if the star is in the upper CCD
fake_df['yIn'] = fake_df.apply(lambda x: x.yIn+2000 if x.extensionIn>1 else x.yIn, axis=1) 
# fake_df.iloc[:5,:16] #print some relevant columns to see if correct

In [54]:
#make some selection cuts
fake_df['selected']=np.where(
    (fake_df['chiOut'] < 2.5) & (fake_df['sharpOut']*fake_df['sharpOut'] <= 0.09) & (fake_df['typeOut'] <= 2) 
    & (fake_df['snr_mag1Out'] >= 5) & (fake_df['flag_mag1Out'] == 0) & (fake_df['snr_mag2Out'] >= 5) 
    & (fake_df['flag_magout'] == 0),True, False)

# fake_df['selected'][:10] #print some relevant columns to see if correct

KeyError: 'snr_mag1Out'

In [12]:
#automation depending on number of filters we have

#initalize columns to store denominator for magnitude conversion
fake_df['vSumCounts'] = 0
fake_df['iSumCounts'] = 0

#find the denominator (sum of counts)
for i in range(numberOfImagesPerFilter):
    vColumnName = 'counts_V%d' %(i+1)
    iColumnName = 'counts_I%d' %(i+1)
    fake_df['vSumCounts'] =  fake_df['vSumCounts'] + fake_df[vColumnName]
    fake_df['iSumCounts'] =  fake_df['iSumCounts'] + fake_df[iColumnName]

In [13]:
#convert to magnitudes
fake_df['dV'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_V/(x.vSumCounts)),3) 
                          if x.selected is True else 99.999, axis=1)
fake_df['dI'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_I/(x.iSumCounts)),3) 
                          if x.selected is True else 99.999, axis=1)

In [14]:
# #compare to old technique to make sure (this is assuming 2 input files in each filter)
# fake_df['dVOG'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_V/(x.counts_V1+x.counts_V2)),3) 
#                           if x.selected is True else 99.999, axis=1)
# fake_df['dIOG'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_I/(x.counts_I1+x.counts_I2)),3) 
#                           if x.selected is True else 99.999, axis=1)

# fake_df['dVdiffTest'] = fake_df['dV'] - fake_df['dVOG']
# fake_df['dIdiffTest'] = fake_df['dI'] - fake_df['dIOG']

In [14]:
# fake_df[fake_df['dVdiffTest'] != 0] #show any place there is a mismatch

In [15]:
# fake_df[fake_df['dIdiffTest'] != 0] #show any place there is a mismatch

In [16]:
#copy columns and create file
fake2_df = fake_df[['mag_V1','mag_I1','inst_vega_V','inst_vega_I','dV','dI']].copy()
fake2_df[:5] #print to have a quick look
fake_cut_file = fake2_df.to_csv('%s.phot.fake2' % (prefix), sep=' ', header = True, index = False)